## Instalación

In [ ]:
!pip install beautifulsoup4 
!pip install tqdm

## Configuración

In [2]:
from bs4 import BeautifulSoup
from requests import get
from time import sleep
import re

from pandas import DataFrame
from tqdm.notebook import tqdm


url = 'https://www.malaga.eu/la-ciudad/agenda/'

### Peticiones web

In [3]:
response = get(url)
response

<Response [200]>

Podemos explorar el contenido de la respuesta con la siguiente función:

In [4]:
text = response.text

def preview_text(large_text: str) -> None:
    """
    Preview the first and last 300 characters of a large text.
    """
    print(f'{len(large_text) = }')
    print(f'{large_text[:300]}\n\n...\n\n{large_text[-300:]}')

preview_text(text)

len(large_text) = 110178
<!DOCTYPE html>
	<html lang="es" itemscope="itemscope" itemtype="http://schema.org/Organization">
	<head>

		<title>Agenda</title>
		<meta charset="UTF-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1.0"> <meta name="descri

...

src="/export/system/modules/com.saga.sagasuite.core.script/resources/respond/1.3.0/respond.js" type="text/javascript" ></script>
	<script src="/export/system/modules/com.saga.sagasuite.core.script/resources/sagasuite/sg-ie7-8.js" type="text/javascript" ></script>
	<![endif]-->

		</body>

	</html>




Es posible que una request como la anterior no nos devuelva el mismo resultado que recibimos utilizando nuestro explorador web. En ese caso, podemos intentar añadir headers que indiquen el navegador web que (teóricamente) estamos utilizando. En este caso, obtenemos el mismo resultado sin añadir estos headers, pero en muchas otras páginas esto no sucede así.

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0'}

response = get(url, headers=headers)

text = response.text

preview_text(text)

len(large_text) = 110178
<!DOCTYPE html>
	<html lang="es" itemscope="itemscope" itemtype="http://schema.org/Organization">
	<head>

		<title>Agenda</title>
		<meta charset="UTF-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1.0"> <meta name="descri

...

src="/export/system/modules/com.saga.sagasuite.core.script/resources/respond/1.3.0/respond.js" type="text/javascript" ></script>
	<script src="/export/system/modules/com.saga.sagasuite.core.script/resources/sagasuite/sg-ie7-8.js" type="text/javascript" ></script>
	<![endif]-->

		</body>

	</html>




Para procesar el html recibido utilizamos BeautifulSoup

In [6]:
soup = BeautifulSoup(response.text)
# soup

Después de haber explorado la página, sabemos que los eventos están en `<div id="agenda">`. Vamos a buscarlo.

In [7]:
events_container = soup.find('div', id='agenda')
events_container

<div class="element parent sg-search advanced sg-search-agenda" id="agenda">
<div class="wrapper">
<header class="headline">
<h2 class="title h2">Agenda<a class="btn btn-gray rounded pull-right" href="https://www.malaga.eu/la-ciudad/dias-festivos/" target="_blank"><i aria-hidden="true" class="fa fa-calendar mr-5"></i>
					Dias Festivos</a>
</h2>
</header>
<form class="sg-search-form sg-search-form-agenda mb-40" id="agd-searchform" method="get" name="agd-searchform" role="search">
<div class="row">
<div class="col-xxs-12 col-sm-5">
<div class="filterheader clearfix mb-30">
<span class="title-element">Calendario</span>
</div>
<div class="datepicker-wrapper datepicker-wrapper-inline datepicker-wrapper-with-caption" data-date-language="es" data-date-multidate="false" data-date-title="Selecciona un día para ver las actividades celebradas" data-date-today-highlight="true" data-date-week-start="1" data-provide="datepicker-inline" data-target-input="fechaDesde" id="calAgd">
</div>
</div>
<div

Dentro, podemos quedarnos con la información de cada evento, que está en un `<article class="media row">`

In [8]:
events = events_container.find_all('article', class_='media')

print(f"Eventos en la página: {len(events)}")
events[:2]

Eventos en la página: 10


[<article class="media row lg-spacing-row">
 <div class="media-object col-xxs-12 col-xs-3">
 <div class="saga-imagen">
 <span class="wrapper-image">
 <img alt="Imagen principal" class="img-responsive" height="320" src="http://www.malaga.eu/inter/visor_contenido/AGDImageDisplayer1/ImagenAgenda48700?id_imagen=48700" width="480"/>
 </span>
 </div>
 </div>
 <div class="media-body container-fluid">
 <h2 class="title-element media-heading h3">
 <a href="/la-ciudad/agenda/detalle-actividad/index.html?id=138182">Manipulador de Alimentos para Celíacos</a>
 </h2>
 <span class="info date-element">
 <span aria-hidden="true" class="pe-7s-date fs30 v-align-m mr-10"></span>
 <time datetime="2025-04-23">
                                     23-04-2025</time>
 </span>
 <span class="info place-element">
 <span aria-hidden="true" class="pe-7s-map-marker fs30 v-align-m mr-10"></span>
 <span>Online</span>
 </span>
 <div class="description">
 <p>
                             El carnet de manipulador de alim

Explorando más, podemos ver que para cada evento tenemos un link a su página de detalle. Vamos a recopilar todas las urls de los eventos.

In [9]:
event_links = [
    event.find("h2").find("a")["href"] for event in events
]

event_links

['/la-ciudad/agenda/detalle-actividad/index.html?id=138182',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139228',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139231',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=138905',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139155',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139328',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139041',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139217',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139148',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139152']

Exploramos lo que aparece en una página de evento

In [10]:
base_url = 'https://www.malaga.eu'

event = event_links[0]

html_e = get(f'{base_url}{event}', headers=headers)

soup = BeautifulSoup(html_e.text)
# Como antes, nos quedamos con la parte que contiene la información del evento
soup = soup.find("article")
soup

<article class="articulo element parent sg-actividad">
<div class="wrapper">
<header class="headline">
<h1 class="title"> Manipulador de Alimentos para Celíacos</h1>
</header>
<div class="contentblock">
<div class="contentblock-texto row">
<div class="media-object image col-xxs-12 col-sm-5 pull-right pos-right">
<div class="saga-imagen">
<a class="wrapper-image zoom-filter" data-title=" Manipulador de Alimentos para Celíacos" href="https://www.malaga.eu/visorcontenido/AGDImageDisplayer1/48700/ImagenAgenda48700" title="Ampliar imagen agenda-l">
<div class="overlay-zoom">
<img alt=" Manipulador de Alimentos para Celíacos" class="img-responsive" height="300" src="https://www.malaga.eu/visorcontenido/AGDImageDisplayer1/48700/ImagenAgenda48700" width="440"/>
<div class="zoom-icon">
<span aria-hidden="true" class="fa fa-expand"></span>
</div>
</div>
</a>
<span class="saga-imagen-pie"><span class="saga-imagen-pie-content"><!-- Pie de imagen --></span></span>
</div>
</div>
<div class="containe

Una función para obtener la información de un evento a partir de esta url:

In [11]:
def get_event_info(event_link: str) -> dict:

    html = get(f'{base_url}{event_link}', headers=headers)
    soup = BeautifulSoup(html.text)
    base = {'event_link': event_link}

    try:
        div_info = soup.find("article")
    # El manejo de errores se puede mejorar (debemos comprobar los errores que podemos recibir)
    except:
        # Si no encontramos el div_info, devolvemos un diccionario con solo el link del evento
        return base

    info = {
        # Detecta el tag <h1>, y nos quedamos con el contenido de "text"
        'title': div_info.h1.text.strip(),
        # Detecta las fechas en el tag <div class="info date-element">:
        "dates": [t.text.strip() for t in div_info.find("div", class_="date-element").find_all("time")],
        # Detecta el único elemento <p> de los datos del evento
        "description": div_info.find("p").text.strip(),
        # Guardamos el texto bajo el título "Ficha informativa". Lo podemos guardar como datos menos estructurados para analizarlos luego
        # Además, limpiamos un poco el texto mediante expresiones regulares.
        # `re.sub(r" {2,}", " ", text)` sustituye los espacios en blanco que van seguidos por un solo espacio
        "info": re.sub(r" {2,}", " ", div_info.find("div", class_="h3").find_next_sibling().get_text()).strip(),
    }
    # Detecta el tag <img class="img-responsive">, y nos quedamos con el contenido de "src"
    if (img_data := div_info.find("img", class_="img-responsive")) is not None:
       info["img_url"] = img_data["src"]

    # Algunos eventos tienen la información de la ubicación
    try:
        location_elem = div_info.find("div", class_="place-element").find("a")
        location = {
            'location_url': location_elem["href"],
            'location_name': location_elem.text.strip()
        }
    except:
        location = {}
    return {**base, **info, **location}


get_event_info(event_links[1])

{'event_link': '/la-ciudad/agenda/detalle-actividad/index.html?id=139228',
 'title': 'Cita con el Tablero. Taller de Ajedrez Infantil. BPM Cristóbal Cuevas',
 'dates': ['23/4/2025', '23/4/2025'],
 'description': 'Dirigido a: niños/as de 6 a 12 años. PREVIA INSCRIPCIÓN. Contenido: Taller de iniciación al ajedrez dirigido a los más pequeños y coordinado por Enrique Díaz Fernández.          2025/04/07 14:04:07',
 'info': 'Evento:\nPlan de Fomento a la Lectura en las Bibliotecas Municipales 2025\nHorario:\n18:30 horas\nTeléfono:\n951 92 6184\nPrecio:\nGratuito\nEmail:\nbiblio.ccuevas@malaga.eu\nDestinatarios:\n\n \n Infantil',
 'img_url': '/export/shared/corporativas/default/agenda-l.png',
 'location_url': '/la-ciudad/instalaciones-y-espacios/detalle-de-la-instalacion/?id=3011',
 'location_name': 'Biblioteca Pública Municipal "Cristóbal Cuevas" (Bailén-Miraflores)'}

Ahora podemos recopilar todos los eventos de la página. Antes recogimos solo los eventos de la primera página. Vamos a recorrer todas las páginas de eventos.

In [12]:
page_idxs = range(1, 7)  # 6 páginas de eventos

# Recorremos las páginas de eventos
# La librería `tqdm` nos permite mostrar una barra de progreso
for page_idx in tqdm(page_idxs):
    url = f'https://www.malaga.eu/la-ciudad/agenda/index.html?mas=true&pageNum={page_idx}'
    html = get(url, headers=headers)
    soup = BeautifulSoup(html.text)
    # Repetimos el proceso anterior para obtener los links de los eventos
    events_container = soup.find('div', id='agenda')
    events = events_container.find_all('article', class_='media')
    event_links += [event.find("h2").find("a")["href"] for event in events]
    # Esperamos un segundo entre peticiones para no saturar el servidor de la página que estamos scrapeando
    sleep(1)

print(f"Eventos encontrados: {len(event_links)}")
# Mostramos los dos primeros
event_links[:2]

  0%|          | 0/6 [00:00<?, ?it/s]

Eventos encontrados: 63


['/la-ciudad/agenda/detalle-actividad/index.html?id=138182',
 '/la-ciudad/agenda/detalle-actividad/index.html?id=139228']

Por último, recorremos las urls de los eventos que hemos encontrado para guardar su información

In [13]:
events_info = []
for event_link in tqdm(event_links):
    events_info.append(get_event_info(event_link))
    sleep(.5)

len(events_info)

  0%|          | 0/63 [00:00<?, ?it/s]

63

Podemos poner la respuesta como una tabla

In [14]:
df = DataFrame(events_info)
df

,event_link,title,dates,description,info,img_url,location_url,location_name
0,/la-ciudad/agenda/detalle-actividad/index.html...,Manipulador de Alimentos para Celíacos,"[23/4/2025, 23/4/2025]",El carnet de manipulador de alimentos es un ce...,Evento:\nIMFE: Jornadas y seminarios \nOtros l...,https://www.malaga.eu/visorcontenido/AGDImageD...,NaN,NaN
1,/la-ciudad/agenda/detalle-actividad/index.html...,Cita con el Tablero. Taller de Ajedrez Infanti...,"[23/4/2025, 23/4/2025]",Dirigido a: niños/as de 6 a 12 años. PREVIA IN...,Evento:\nPlan de Fomento a la Lectura en las B...,/export/shared/corporativas/default/agenda-l.png,/la-ciudad/instalaciones-y-espacios/detalle-de...,"Biblioteca Pública Municipal ""Cristóbal Cuevas..."
2,/la-ciudad/agenda/detalle-actividad/index.html...,Día del Libro: Irene Vallejo.,"[23/4/2025, 23/4/2025]","Charlará con la autora Ana Cabello, gerente de...",Evento:\nFundación Rafael Pérez Estrada\nHorar...,/export/shared/corporativas/default/agenda-l.png,/la-ciudad/instalaciones-y-espacios/detalle-de...,Centre Pompidou Málaga
3,/la-ciudad/agenda/detalle-actividad/index.html...,"TEATRO ""EL CIRCO DE LAS EMOCIONES"" C.E.I.P. PR...","[23/4/2025, 23/4/2025]",Representación teatral correspondiente al proy...,Evento:\nLA CAJA BLANCA - Auditorio 2025\nHora...,NaN,/la-ciudad/instalaciones-y-espacios/detalle-de...,La Caja Blanca
4,/la-ciudad/agenda/detalle-actividad/index.html...,Taller de Poesía Grupo 21. B.P.M. Manuel Altol...,"[23/4/2025, 23/4/2025]",Dirigido. Público adulto. PREVIA INSCRIPCIÓN\r...,Evento:\nPlan de Fomento a la Lectura en las B...,/export/shared/corporativas/default/agenda-l.png,/la-ciudad/instalaciones-y-espacios/detalle-de...,"Biblioteca Pública Municipal ""Manuel Altolagui..."
...,...,...,...,...,...,...,...,...
58,/la-ciudad/agenda/detalle-actividad/index.html...,Taller de manualidades,"[4/10/2024, 4/10/2025]",Taller de manualidades donde se practican y ap...,Evento:\nACTIVIDADES AMFAEM 2024-25\nOtros lug...,/export/shared/corporativas/default/agenda-l.png,NaN,NaN
59,/la-ciudad/agenda/detalle-actividad/index.html...,Terapia de grupo,"[4/10/2024, 4/10/2025]",Sesiones de Terapia de grupo en relación a asp...,Evento:\nACTIVIDADES AMFAEM 2024-25\nOtros lug...,/export/shared/corporativas/default/agenda-l.png,NaN,NaN
60,/la-ciudad/agenda/detalle-actividad/index.html...,"Simed, Salón Inmobiliario del Mediterráneo","[13/11/2025, 15/11/2025]",El evento líder del sector residencial especia...,Evento:\nSimed ¿ Salón Inmobiliario del Medite...,https://www.malaga.eu/visorcontenido/AGDImageD...,/la-ciudad/instalaciones-y-espacios/detalle-de...,Palacio de Ferias y Congresos de Málaga
61,/la-ciudad/agenda/detalle-actividad/index.html...,Picasso. Imágenes cerámicas,"[14/6/2024, 6/10/2025]",La cerámica de Picasso ha sido injustamente vi...,Evento:\nMuseo Casa Natal Picasso\nHorario:\nD...,/export/shared/corporativas/default/agenda-l.png,/la-ciudad/instalaciones-y-espacios/detalle-de...,Museo Casa Natal Picasso


In [15]:
df.to_csv('malaga_events_20250424.csv', index=False)